In [ ]:
import mysql.connector
import tkinter as tk
from tkinter import messagebox

# Connection:
def create_connection():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="12345",
            database="inventory_db"
        )
        return conn
    except mysql.connector.Error as err:
        messagebox.showerror("Connection Error", f"Error: {err}")
        return None

# Table creation:
def create_table():
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''USE En-in''')
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS inventory (
            id INT AUTO_INCREMENT PRIMARY KEY,
            item_name VARCHAR(255),
            quantity INT,
            price DECIMAL(10, 2)
        );
        """)
        conn.commit()
        messagebox.showinfo("Success", "Table created successfully!")
        cursor.close()
        conn.close()

# Add item:
def add_item(item_name, quantity, price):
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        insert_query = "INSERT INTO inventory (item_name, quantity, price) VALUES (%s, %s, %s)"
        cursor.execute(insert_query, (item_name, quantity, price))
        conn.commit()
        messagebox.showinfo("Success", f"Item '{item_name}' added successfully!")
        cursor.close()
        conn.close()

# Update Item:
def update_item(item_id, new_name, new_quantity, new_price):
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        update_query = """
        UPDATE inventory
        SET item_name = %s, quantity = %s, price = %s
        WHERE id = %s
        """
        cursor.execute(update_query, (new_name, new_quantity, new_price, item_id))
        conn.commit()
        messagebox.showinfo("Success", f"Item with ID {item_id} updated successfully!")
        cursor.close()
        conn.close()

# Add Column:
def add_column(column_name, column_type):
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        alter_query = f"ALTER TABLE inventory ADD COLUMN {column_name} {column_type}"
        cursor.execute(alter_query)
        conn.commit()
        messagebox.showinfo("Success", f"Column '{column_name}' added successfully!")
        cursor.close()
        conn.close()

# View inventory:
def view_inventory():
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM inventory")
        result = cursor.fetchall()
        if result:
            inventory_list = "\n".join([f"ID: {row[0]}, Name: {row[1]}, Quantity: {row[2]}, Price: {row[3]}" for row in result])
            messagebox.showinfo("Inventory List", inventory_list)
        else:
            messagebox.showinfo("No Data", "No items found in the inventory.")
        cursor.close()
        conn.close()

# Max stock (Updated function):
def max_stock():
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        cursor.execute("""
        SELECT item_name, quantity
        FROM inventory
        ORDER BY quantity DESC
        LIMIT 1
        """)
        result = cursor.fetchone()
        if result:
            messagebox.showinfo("Max Stock", f"Item with Maximum Stock: {result[0]} with {result[1]} items")
        else:
            messagebox.showinfo("No Data", "No items found.")
        cursor.close()
        conn.close()

# Count Products:
def count_products():
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM inventory")
        result = cursor.fetchone()
        messagebox.showinfo("Total Products", f"Total number of products: {result[0]}")
        cursor.close()
        conn.close()

# Join Tables:
def join_tables():
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        cursor.execute("""
        SELECT inventory.item_name, sales.sale_date, sales.sale_price
        FROM inventory
        JOIN sales ON inventory.id = sales.item_id
        """)
        result = cursor.fetchall()
        if result:
            joined_data = "\n".join([f"Item: {row[0]}, Sale Date: {row[1]}, Sale Price: {row[2]}" for row in result])
            messagebox.showinfo("Joined Data", joined_data)
        else:
            messagebox.showinfo("No Data", "No sales records found.")
        cursor.close()
        conn.close()

# (GUI) Graphic User Interface:
class InventoryApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Inventory Management System")
        self.root.geometry("600x400")
        self.root.config(bg="#f4f4f4")  # Background color for the window
        self.create_widgets()

    def create_widgets(self):
        self.label_name = tk.Label(self.root, text="Item Name", bg="#f4f4f4")
        self.label_name.pack()

        self.entry_name = tk.Entry(self.root, bg="#e0e0e0")
        self.entry_name.pack()

        self.label_quantity = tk.Label(self.root, text="Quantity", bg="#f4f4f4")
        self.label_quantity.pack()

        self.entry_quantity = tk.Entry(self.root, bg="#e0e0e0")
        self.entry_quantity.pack()

        self.label_price = tk.Label(self.root, text="Price", bg="#f4f4f4")
        self.label_price.pack()

        self.entry_price = tk.Entry(self.root, bg="#e0e0e0")
        self.entry_price.pack()

        self.label_id = tk.Label(self.root, text="Item ID (for update)", bg="#f4f4f4")
        self.label_id.pack()

        self.entry_id = tk.Entry(self.root, bg="#e0e0e0")
        self.entry_id.pack()

        self.button_create = tk.Button(self.root, text="Create Table", command=create_table, bg="#007bff", fg="white")
        self.button_create.pack()

        self.button_add = tk.Button(self.root, text="Add Item", command=self.add_item, bg="#28a745", fg="white")
        self.button_add.pack()

        self.button_update = tk.Button(self.root, text="Update Item", command=self.update_item, bg="#ffc107", fg="white")
        self.button_update.pack()

        self.button_view = tk.Button(self.root, text="View Inventory", command=view_inventory, bg="#17a2b8", fg="white")
        self.button_view.pack()

        self.button_max_stock = tk.Button(self.root, text="Max Stock", command=max_stock, bg="#dc3545", fg="white")
        self.button_max_stock.pack()

        self.button_count = tk.Button(self.root, text="Count Products", command=count_products, bg="#6c757d", fg="white")
        self.button_count.pack()

        self.button_join = tk.Button(self.root, text="View Joined Data", command=join_tables, bg="#343a40", fg="white")
        self.button_join.pack()

        # Clear button to reset the entries
        self.button_clear = tk.Button(self.root, text="Clear Entries", command=self.clear_entries, bg="#6c757d", fg="white")
        self.button_clear.pack()

    def add_item(self):
        item_name = self.entry_name.get()
        quantity = int(self.entry_quantity.get())
        price = float(self.entry_price.get())
        add_item(item_name, quantity, price)
        self.clear_entries()  # Clear after adding item

    def update_item(self):
        item_id_str = self.entry_id.get()

        # Check if the ID field is empty
        if not item_id_str:
            messagebox.showerror("Input Error", "Please enter the Item ID to update.")
            return

        try:
            item_id = int(item_id_str)  # Try to convert it to integer
        except ValueError:
            messagebox.showerror("Input Error", "Invalid Item ID. Please enter a valid number.")
            return

        new_name = self.entry_name.get()
        new_quantity = int(self.entry_quantity.get())
        new_price = float(self.entry_price.get())
        update_item(item_id, new_name, new_quantity, new_price)
        self.clear_entries()  # Clear after updating item

    def clear_entries(self):
        """Clear all entry fields."""
        self.entry_name.delete(0, tk.END)
        self.entry_quantity.delete(0, tk.END)
        self.entry_price.delete(0, tk.END)
        self.entry_id.delete(0, tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = InventoryApp(root)
    root.mainloop()
